EXAMPLE FOR API BASED TEXT MINING

importing the reqires libraries

In [ ]:
from tweepy import API 
from tweepy import Cursor
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from textblob import TextBlob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re


Twitter API Keys as follows

In [ ]:
#API KEYS PROVIDED BY TWITTER
API_KEY = "pse5QOaNqCk1TKz7cBR5UdiMA"
API_SECRET = "hYOkDIae7FBArDQJ2pEHfN6h2UVvEXqjO58zAyzNsL5JUsFLNj"
ACCESS_TOKEN = "1198200370440196096-1e0lZUiWXAxG8GEY4qjHSGX8cJh0IW"
ACCESS_TOKEN_SECRET = "nFCx1dwiYfkrStff88F92Du1U0nyCYMycPH7hfmmzycjs"

Datastructure Creation to accept data from the website

In [ ]:
class TwitterClient():
  def __init__(self, twitter_user=None):
    self.auth = TwitterAuthenticator().authenticate_twitter_app()
    self.twitter_client = API(self.auth)
    self.twitter_user = twitter_user

  def get_twitter_client_api(self):
      return self.twitter_client

  def get_user_timeline_tweets(self, num_tweets):
      tweets = []
      for tweet in Cursor(self.twitter_client.user_timeline, id=self.twitter_user).items(num_tweets):
          tweets.append(tweet)
      return tweets

  def get_friend_list(self, num_friends):
      friend_list = []
      for friend in Cursor(self.twitter_client.friends, id=self.twitter_user).items(num_friends):
          friend_list.append(friend)
      return friend_list

  def get_home_timeline_tweets(self, num_tweets):
      home_timeline_tweets = []
      for tweet in Cursor(self.twitter_client.home_timeline, id=self.twitter_user).items(num_tweets):
          home_timeline_tweets.append(tweet)
      return home_timeline_tweets

Authenticator class to enable the tunnel

In [ ]:
class TwitterAuthenticator():
  def authenticate_twitter_app(self):
      auth = OAuthHandler(API_KEY, API_SECRET)
      auth.set_access_token(ACCESS_TOKEN,ACCESS_TOKEN_SECRET)
      return auth

Text Preprocessing Processing

In [ ]:
class TwitterStreamer():
  def __init__(self):
    self.twitter_autenticator = TwitterAuthenticator()    

  def stream_tweets(self, fetched_tweets_filename, hash_tag_list):
    # This handles Twitter authetification and the connection to Twitter Streaming API
    listener = TwitterListener(fetched_tweets_filename)
    auth = self.twitter_autenticator.authenticate_twitter_app() 
    stream = Stream(auth, listener)

    # This line filter Twitter Streams to capture data by the keywords: 
    stream.filter(track=hash_tag_list)


Listener class to print the tweets

In [ ]:
# # # # TWITTER STREAM LISTENER # # # #
class TwitterListener(StreamListener):
    """
    This is a basic listener that just prints received tweets to stdout.
    """
    def __init__(self, fetched_tweets_filename):
        self.fetched_tweets_filename = fetched_tweets_filename

    def on_data(self, data):
        try:
            print(data)
            with open(self.fetched_tweets_filename, 'a') as tf:
                tf.write(data)
            return True
        except BaseException as e:
            print("Error on_data %s" % str(e))
        return True
          
    def on_error(self, status):
        if status == 420:
            # Returning False on_data method in case rate limit occurs.
            return False
        print(status)

Sentiment Analyser

In [ ]:
class TweetAnalyzer():
    """
    Functionality for analyzing and categorizing content from tweets.
    """
 
    def clean_tweet(self, tweet):
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())
 
    def analyze_sentiment(self, tweet):
        analysis = TextBlob(self.clean_tweet(tweet))
        
        if analysis.sentiment.polarity > 0:
            return 1
        elif analysis.sentiment.polarity == 0:
            return 0
        else:
            return -1
 
    def tweets_to_data_frame(self, tweets):
        df = pd.DataFrame(data=[tweet.text for tweet in tweets], columns=['tweets'])
 
        df['id'] = np.array([tweet.id for tweet in tweets])
        df['len'] = np.array([len(tweet.text) for tweet in tweets])
        df['date'] = np.array([tweet.created_at for tweet in tweets])
        df['source'] = np.array([tweet.source for tweet in tweets])
        df['likes'] = np.array([tweet.favorite_count for tweet in tweets])
        df['retweets'] = np.array([tweet.retweet_count for tweet in tweets])
 
        return df

Main block to pull data from twitter

In [ ]:
if __name__ == '__main__':

    twitter_client = TwitterClient()
    tweet_analyzer = TweetAnalyzer()

    api = twitter_client.get_twitter_client_api()

    tweets = api.user_timeline(screen_name="the_hindu", count=1000)

    df = tweet_analyzer.tweets_to_data_frame(tweets)
    df['sentiment'] = np.array([tweet_analyzer.analyze_sentiment(tweet) for tweet in df['tweets']])

    print(df)

                                                tweets  ...  sentiment
0    Experts flag lack of published data on perform...  ...          0
1    Prime Minister #NarendraModi will not attend t...  ...          0
2    With experts warning of a third wave as well, ...  ...          1
3    Shifting of a COVID-19 patient from Delhi to C...  ...          1
4    At present, Goods and Services Tax is levied a...  ...          0
..                                                 ...  ...        ...
195  U.S. regulators authorised #Pfizer and BioNTec...  ...          1
196  Mr. Johnson, who was buoyed by successful loca...  ...          1
197  More than 100 churches have signed up to host ...  ...          1
198  Pakistan was put on the grey list by the Paris...  ...          1
199  CITU leaders Ch. Ammannaidu urged Collector J....  ...          0

[200 rows x 8 columns]


Porter Stemmer

In [ ]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()

Porter - Tokenization

In [ ]:
def porter_stemmer_tokenizer(text):
  return[porter.stem(word) for word in text.split()]

Porter - Vectorization

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(strip_accents = None, lowercase = False, tokenizer = porter_stemmer_tokenizer, use_idf = True, norm = 'l2', smooth_idf = True)

Porter - Categorizing the target and dependent

In [ ]:
X = tfidf.fit_transform(df.tweets)
Y = df.sentiment.values

Porter - Splitting the test and Train data

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test =train_test_split(X, Y, random_state=1, test_size=0.1, shuffle= False)

Porter - Performing Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegressionCV
logitCV = LogisticRegressionCV( cv=5, scoring='accuracy', max_iter=500 ).fit(X_train, Y_train)

Accuracy score of Porter stemmer

In [ ]:
logitCV.score(X_test, Y_test)

0.5

Lancaster stemmer

In [ ]:
from nltk.stem import LancasterStemmer
lancaster=LancasterStemmer()

Lancaster - Tokenization

In [ ]:
def lancaster_stemmer_tokenizer(text):
  return[lancaster.stem(word) for word in text.split()]

Lancaster - Vectorization

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(strip_accents = None, lowercase = False, tokenizer = lancaster_stemmer_tokenizer, use_idf = True, norm = 'l2', smooth_idf = True)

Lancaster - Categorizing the target and dependent

In [ ]:
X = tfidf.fit_transform(df.tweets)
Y = df.sentiment.values

Lancaster - Splitting the test and Train data

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test =train_test_split(X, Y, random_state=1, test_size=0.1, shuffle= False)

Lancaster - Performing Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegressionCV
logitCV = LogisticRegressionCV( cv=5, scoring='accuracy', max_iter=500 ).fit(X_train, Y_train)

Accuracy score of Lancaster stemmer

In [ ]:
logitCV.score(X_test, Y_test)

0.45